<a href="https://colab.research.google.com/github/pashtetXIII13/Annatation/blob/master/notebooks/YOLO_NAS_Pretrained_Export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
! pip install git+https://github.com/Deci-AI/super-gradients.git --break-system-packages

  Cloning https://github.com/Deci-AI/super-gradients.git to /tmp/pip-req-build-td07how3
  Running command git clone --filter=blob:none --quiet https://github.com/Deci-AI/super-gradients.git /tmp/pip-req-build-td07how3
  Resolved https://github.com/Deci-AI/super-gradients.git to commit e0ccacf8868ffa1296fa4f8407c03d2bc227312c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached boto3-1.37.8-py3-none-any.whl.metadata (6.6 kB)
  Using cached torchmetrics-0.8.0-py3-none-any.whl.metadata (20 kB)
  Using cached hydra_core-1.3.2-py3-none-any.whl.metadata (5.5 kB)
  Using cached onnxruntime-1.20.1-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (4.5 kB)
  Using cached onnx-1.15.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (15 kB)
  Using cached pip_tools-7.4.1-py3-none-any.whl.metadata (26 kB)
  Using cached einops-0.3.2-py3-none-any.whl.metadata (10 kB

In [1]:
! sed -i 's/sghub.deci.ai/sg-hub-nv.s3.amazonaws.com/' /usr/local/lib/python3.11/dist-packages/super_gradients/training/pretrained_models.py
! sed -i 's/sghub.deci.ai/sg-hub-nv.s3.amazonaws.com/' /usr/local/lib/python3.11/dist-packages/super_gradients/training/utils/checkpoint_utils.py

In [2]:
from super_gradients.common.object_names import Models
from super_gradients.conversion import DetectionOutputFormatMode
from super_gradients.training import models

model = models.get(Models.YOLO_NAS_S, pretrained_weights="coco")

The console stream is logged into /root/sg_logs/console.log


[2025-03-07 17:38:21] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it
[2025-03-07 17:38:33] INFO - utils.py - NumExpr defaulting to 2 threads.
/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.5 (you have 1.4.24). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
[2025-03-07 17:38:50] INFO - env_sanity_check.py - Library check is not supported when super_gradients installed through "git+https://github.com/..." command
[2025-03-07 17:38:51] WARNING - checkpoint_utils.py - :warning: The pre-trained models provided by SuperGradients may have their own licenses or terms and conditions derived from the dataset used for pre-training.
 It is your responsibility to determine whether you have permission to use the models f

In [3]:
# export the model for compatibility with Frigate

model.export("yolo_nas_s.onnx",
             output_predictions_format=DetectionOutputFormatMode.FLAT_FORMAT,
             max_predictions_per_image=20,
             confidence_threshold=0.4,
             input_image_shape=(320,320),
            )

/usr/local/lib/python3.11/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

Collecting onnx_graphsurgeon==0.3.27

Downloading https://developer.download.nvidia.com/compute/redist/onnx-graphsurgeon/onnx_graphsurgeon-0.3.27-py2.py3-none-any.whl (42 kB)

Output()

...

Requirement already satisfied: protobuf>=3.20.2 in /usr/local/lib/python3.11/dist-packages (from onnx->onnx_graphsurgeon==0.3.27) (4.25.6)

Installing collected packages: onnx_graphsurgeon

Successfully installed onnx_graphsurgeon-0.3.27

/usr/local/lib/python3.11/dist-packages/torch/onnx/symbolic_opset9.py:5385: UserWarning: Exporting aten::index operator of advanced indexing in opset 17 is achieved by combination of multiple ONNX operators, including Reshape, Transpose, Concat, and Gather. If indices include negative values, the exported graph will produce incorrect results.
  warnings.warn(


Model exported successfully to yolo_nas_s.onnx
Model expects input image of shape [1, 3, 320, 320]
Input image dtype is torch.uint8
Exported model already contains preprocessing (normalization) step, so you don't need to do it manually.
Preprocessing steps to be applied to input image are:
Sequential(
  (0): CastTensorTo(dtype=torch.float32)
  (1): ApplyMeanStd(mean=[0.], scale=[255.])
)

Exported model contains postprocessing (NMS) step with the following parameters:
    num_pre_nms_predictions=1000
    max_predictions_per_image=20
    nms_threshold=0.7
    confidence_threshold=0.4
    output_predictions_format=DetectionOutputFormatMode.FLAT_FORMAT

Exported model is in ONNX format and can be used with ONNXRuntime
To run inference with ONNXRuntime, please use the following code snippet:

    import onnxruntime
    import numpy as np
    session = onnxruntime.InferenceSession("yolo_nas_s.onnx", providers=["CUDAExecutionProvider", "CPUExecutionProvider"])
    inputs = [o.name for o in s

In [4]:
from google.colab import files

files.download('yolo_nas_s.onnx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>